In [1]:
import sys
import os
import transformers
from transformers import pipeline
import json
import torch
import numpy as np
from torch.utils.data import DataLoader , Dataset

/mnt/DGX01/Personal/slndir/evaluation/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
sys.path.append("../")

In [3]:
#loading in the JSON list of dictionaries
data = json.load(open("../data/geneset_dict.json"))

In [4]:
print(f"Number of items in geneset_dict: {len(data)}")

Number of items in geneset_dict: 43829


In [5]:
#load in llama model
model_id = "meta-llama/Llama-3.1-70B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [6]:
query = """
Format this data into several question and answer pairs. Use the information from the keys to make multiple questions, 
and the information from the values to make the answer. Prioritize gene id first, then gene name, then aliases, then other names, and finally description.
Format multiple questions to resemble a user query. Make sure you use every key for its own separate question, 
and number the questions. Keep the description as it is. In this case 'aliases' mean synonym symbols for the gene, while 'other names' mean other names for the gene.
Adress every key, but ignore any objects with ['N/A'] as their value. 
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

"""

In [7]:
#function to generate QA pairs from the model
def generate_qa_pairs(sample):
    messages = [
        {"role": "system", "content": query},
        {"role": "user", "content": str(sample)},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]

In [8]:
#to work GPUs parralel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
# Convert each item into a string prompt with the base instructions
prompts = [
    f"{base_prompt}\n{json.dumps(item, indent=2)}"
    for item in filtered_data[:12]  #adjust number as needed
]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [9]:
print(outputs)

[[{'generated_text': "{'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'}"}], [{'generated_text': "{'Symbol': 'A1BG-AS1', 'Gene_ID': 'ENSG00000268895', 'Name': 'A1BG antisense RNA 1', 'Aliases': ['A1BG-AS', 'A1BGAS', 'NCRNA00181'], 'Other names': ['A1BG antisense RNA (non-protein coding)', 'A1BG antisense RNA 1 (non-protein coding)'], 'Description': 'N/A'} {'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'Alpha-1-B glycoprotein', 'Aliases': ['A1B', 'HYST1037'], 'Other names': 'Alpha-1-B glycoprotein precursor', 'Description': 'The protein encoded by

In [11]:
# remove the <question> and <answer> tags from the generated text: copied from chatgpt
final_qa_pairs = []
for result in outputs:
    # Access the 'generated_text' field, which is a string, from the result
    qa_text = result.get('generated_text', '')
    
    # Split the result by <question> and <answer> tags, extract them
    questions_answers = qa_text.split('<question>')
    pair = {}
    for q_a in questions_answers[1:]:
        question_end = q_a.find('</question>')
        answer_end = q_a.find('</answer>')
        question = q_a[:question_end].strip()
        answer = q_a[question_end + len('</question>'):answer_end].strip()
        pair[question] = answer
    final_qa_pairs.append(pair)

with open('qa_pairs.json', 'w') as json_file:
    json.dump(final_qa_pairs, json_file, indent=4)
# print out all the results
print("JSON file saved successfully")

JSON file saved successfully


In [2]:


# Setup: Add parent directory to path
sys.path.append("../")

# Environment for multi-GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# Load dataset: list of dicts
data = json.load(open("../data/geneset_dict.json"))

# Load model pipeline
model_id = "meta-llama/Llama-3.1-70B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Instruction prompt for LLM
base_prompt = """
Format this data into several question and answer pairs. Use the information from the keys to make multiple questions, 
and the information from the values to make the answer. Prioritize gene id first, then gene name, then aliases, then other names, and finally description.
Format multiple questions to resemble a user query. Make sure you use every key for its own separate question, 
and number the questions. Keep the description as it is. In this case 'aliases' mean synonym symbols for the gene, while 'other names' mean other names for the gene.
Address every key, but ignore any objects with ['N/A'] as their value. 
Provide the pair in this format: <question>question here</question><answer>answer here</answer>
"""

# Only use items that don’t have 'N/A' as Description
filtered_data = [item for item in data if item.get("Description") != "N/A"]



# Generate outputs in batch
outputs = pipe(prompts, max_new_tokens=256)



Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AttributeError: 'list' object has no attribute 'get'

In [3]:
final_qa_pairs = []
for result in outputs:
    for res in result:
        qa_text = res.get('generated_text', '')
        questions_answers = qa_text.split('<question>')
        pair = {}
        for q_a in questions_answers[1:]:
            question_end = q_a.find('</question>')
            answer_end = q_a.find('</answer>')
            if question_end != -1 and answer_end != -1:
                question = q_a[:question_end].strip()
                answer = q_a[question_end + len('</question>'):answer_end].strip()
                pair[question] = answer
        final_qa_pairs.append(pair)

# Save result
with open('qa_pairs.json', 'w') as json_file:
    json.dump(final_qa_pairs, json_file, indent=4)

print("JSON file saved successfully")

JSON file saved successfully
